# FORMULACIÓN DEL MODELO DE OPTIMIZACIÓN

La formulación del modelo se basa en la representación matemática de las relaciones entre actores de la red planteada. Para esto se formula un algoritmos de programación lineal de enteros mixtos (MILP) que contemple el problema de asignación de cantidad optima de cada tipo de queso y operación de agentes dentro de la red, para llegar a la modelación definitiva se lleva a cabo las distintas pruebas de funcionalidad con el fin de validad cada una de las restricciones, supuestos y variables de salida, con el fin de verificar la validez de los resultados. 

## Modelo de optimización

### 1. Importa la librería 

In [62]:
# Instalando biblioteca pyomo y GLPK
# Instalando Pyomo
# pip3 install pyomo
# pip3 install pandas

In [63]:

import pandas as pd
from pyomo.environ import*
import pyomo.environ as pyo

### 2. Crea un modelo de Pyomo 

In [64]:
# Crear un modelo abstracto con Pyomo
modelo = pyo.AbstractModel()

### 3. Cargamos los parametros

In [65]:
# Cargar datos desde archivos CSV
# Asumiendo que tienes archivos CSV llamados 'costo_produccion.csv', 'costo_fijo_acopio.csv', etc.
# Ajusta los nombres de archivos según tus necesidades

productores_df = pd.read_csv('Productores.csv', index_col='Productores')
centros_de_acopios_df = pd.read_csv('CAcopios.csv', index_col='CAcopios')
clientes_df = pd.read_csv('Clientes.csv', index_col='Clientes')
productos_df = pd.read_csv('Productos.csv', index_col='Productos')
periodo_de_tiempo_df = pd.read_csv('Tiempo.csv', index_col='Tiempo')

costo_produccion_df = pd.read_csv('CostoProduccion.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_fijo_acopio_df = pd.read_csv('CostoFijoAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_var_acopio_df = pd.read_csv('CostoVarAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_inv_acopio_df = pd.read_csv('CostoInvAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_prod_acopio_df = pd.read_csv('CostoTransProdAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_acopio_clie_df = pd.read_csv('CostoTransAcopioClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
demanda_clie_df = pd.read_csv('DemandaClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig') 
cap_productor_df = pd.read_csv('CapProductor.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
cap_almacenamientoCA_df = pd.read_csv('CapAlmacenamientoCA.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
productos_dinsatisfecha_df = pd.read_csv('PDinsatisfecha.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  

# Convertir los DataFrames a diccionarios
costo_produccion_dict = {(row['Productos'], row['Productores'], row['Tiempo']): row['CostoProduccion'] for _, row in costo_produccion_df.iterrows()}
costo_fijo_acopio_dict = {(row['CAcopios'], row['Tiempo']): row['CostoFijoAcopio'] for _, row in costo_fijo_acopio_df.iterrows()}
costo_var_acopio_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CostoVarAcopio'] for _, row in costo_var_acopio_df.iterrows()}
costo_inv_acopio_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CostoInvAcopio'] for _, row in costo_inv_acopio_df.iterrows()}
costo_trans_prod_acopio_dict = {(row['Productos'], row['Productores'], row['CAcopios'], row['Tiempo']): row['CostoTransProdAcopio'] for _, row in costo_trans_prod_acopio_df.iterrows()}
costo_trans_acopio_clie_dict = {(row['Productos'], row['Productores'], row['Clientes'], row['Tiempo']): row['CostoTransAcopioClie'] for _, row in costo_trans_acopio_clie_df.iterrows()}
demanda_clie_dict = {(row['Productos'], row['Clientes'], row['Tiempo']): row['DemandaClie']  for _, row in demanda_clie_df.iterrows()}
cap_productor_dict = {(row['Productos'], row['Productores'], row['Tiempo']): row['CapProductor']  for _, row in cap_productor_df.iterrows()}
cap_almacenamientoCA_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CapAlmacenamientoCA']  for _, row in cap_almacenamientoCA_df.iterrows()}
productos_dinsatisfecha_dict = {(row['Productos'], row['Tiempo']): row['PDinsatisfecha']  for _, row in productos_dinsatisfecha_df.iterrows()}


In [66]:
# Antes de la creación de los conjuntos, agrega estas líneas
print("Claves de los conjuntos:")
print("Productores:", productores_df.index)
print("Centros de acopios:", centros_de_acopios_df.index)
print("Clientes:", clientes_df.index)
print("Productos:", productos_df.index)
print("Periodo de tiempo:", periodo_de_tiempo_df.index)

# Luego de la creación de los diccionarios, agrega estas líneas
print("\nClaves de los diccionarios:")
print("CostoProduccion:", costo_produccion_dict.keys())
print("CostoFijoAcopio:", costo_fijo_acopio_dict.keys())
print("CostoVarAcopio:", costo_var_acopio_dict.keys())
print("CostoInvAcopio:", costo_inv_acopio_dict.keys())
print("CostoTransProdAcopio:", costo_trans_prod_acopio_dict.keys())
print("CostoTransAcopioClie:", costo_trans_acopio_clie_dict.keys())
print("DemandaClie:", demanda_clie_dict.keys())
print("CapProductor:", cap_productor_dict.keys())
print("CapAlmacenamientoCA:", cap_almacenamientoCA_dict.keys())

Claves de los conjuntos:
Productores: Index(['LUIS MANUEL MEDINA TORRES', 'JUAN LORENZO DUARTE',
       'SEBASTIAN MANUEL ALCALA CANTILLO', 'WILSON ALFREDO IBARRA DELUQUE',
       'ENDER NICOLAS CABRALES', 'NATIVIDAD RIVEIRA', 'MARITZA LEON VANEGAS',
       'OLGA DAZA IBARRA', 'CARLOS ALBERTO GUTIERREZ',
       'GUSTAVO CONTRERAS PEREZ', 'RAFAEL VALMIRO PINTO SAURITH',
       'CARLOS ALBERTO ACOSTA ULBINA', 'NEVIS FLOREZ TORRES',
       'FLORENTINO GUERRA ANAYA', 'ALVARO DE JESUS GUERRA AMAYA',
       'JUAN BARRIOS', 'NEFER ORTIZ', 'YOLMIS AMAYA ORTIZ',
       'ANTONIO MARÍA PACHECO', 'ANDRES DE JESUS LLERENA PEREZ',
       'MANUEL ANTONIO GOMEZ FLOREZ', 'MANUEL JOSE BORRERO SANCHEZ',
       'CARMEN RODRIGUEZ GAMEZ', 'VICTOR DANIEL FUENTES SAJUTH',
       'CARLOS RAFAEL GUERRA URIBIA', 'ALEXANDER VASQUEZ ROMERO',
       'HERMES VAZQUEZ', 'LUIS AGUSTIN MAESTRE', 'MERITH MURGAS',
       'DALHIS MARIA HERNANDEZ BARRETO', 'MIRIAM MIRANDA',
       'EDILBERTO DAZA CORDOBA', 'OSCAR GAMEZ ARIZ

### 4. Conjuntos

| Término | Descripción                    |
|---------|--------------------------------|
| I       | Conjunto de productores        |
| J       | Conjunto de centros de acopio  |
| K       | Conjunto de clientes           |
| P       | Conjunto de productos          |
| T       | Conjunto de periodos de tiempo |


In [67]:

modelo.productores = pyo.Set(initialize=productores_df.index)  # Conjunto I de productores
modelo.centros_de_acopios = pyo.Set(initialize=centros_de_acopios_df.index)  # Conjunto J de centros de acopio
modelo.clientes = pyo.Set(initialize=clientes_df.index)  # Conjunto K de clientes
modelo.productos = pyo.Set(initialize=productos_df.index)  # Conjunto P de productos
modelo.periodo_de_tiempo = pyo.Set(initialize=periodo_de_tiempo_df.index)  # Conjunto T de periodos de tiempo

### 5. Parámetros 

| Término                       | Descripción                                                                       | Unidades    |
|-------------------------------|-----------------------------------------------------------------------------------|-------------|
| Costos de producción          |                                                                               |             |
| $CostoProducción_{pit}$        | Costo de producción del producto p en el productor i en el periodo de tiempo t    | [$/kg]      |
| Costos de operación           |                                                                               |             |
| $CostoFijoAcopi_{jt}$         | Costo fijo del centro de acopio j en el periodo de tiempo t                        | [$]         |
| $CostoVarAcopio_{pjt}$         | Costo variable del producto p en el centro de acopio j en el periodo de tiempo t  | [$/kg]      |
| Costos de inventario          |                                                                               |             |
| $CostoInvAcopio_{pjt}$         | Costo de inventario del producto p en el centro de acopio j en el periodo de tiempo t | [$/kg]   |
| Costos de transporte          |                                                                               |             |
| $CostoTransProdAcopi_{pijt}$  | Costo de transporte del producto p desde el productor i hasta el centro de acopio j en el periodo de tiempo t | [$/kg] |
| $CostoTransAcopioClie_{pjkt}$  | Costo de transporte del producto p desde el centro de acopio j hasta el cliente k en el periodo de tiempo t | [$/kg] |
| Demanda                       |                                                                               |             |
| $DemandaClie_{pkt}$            | Demanda del producto p en el cliente k en el periodo de tiempo t                  | [kg]        |
| Capacidades                   |                                                                               |             |
| $CapProductor_{pit}$           | Capacidad de suministro del producto p en el productor i en el periodo de tiempo t | [kg]        |
| $CapAlmacenamientoCA_{pjt}$    | Capacidad de almacenamiento del producto p en el centro de acopio j en el periodo de tiempo t | [kg]   |
| $PDinsatisfecha_{pt}$          | Costo de la demanda insatisfecha del producto p en el periodo de tiempo t         | [$/kg]      |


In [68]:

modelo.CostoProduccion = pyo.Param(modelo.productos, modelo.productores, modelo.periodo_de_tiempo, initialize = costo_produccion_dict)
modelo.CostoFijoAcopio = pyo.Param(modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_fijo_acopio_dict)
modelo.CostoVarAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_var_acopio_dict)
modelo.CostoInvAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_inv_acopio_dict)
modelo.CostoTransProdAcopio = pyo.Param(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_trans_prod_acopio_dict)
modelo.CostoTransAcopioClie = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, initialize = costo_trans_acopio_clie_dict)
modelo.DemandaClie = pyo.Param(modelo.productos, modelo.clientes, modelo.periodo_de_tiempo, initialize = demanda_clie_dict)
modelo.CapProductor = pyo.Param(modelo.productos, modelo.productores, modelo.periodo_de_tiempo, initialize = cap_productor_dict)
modelo.CapAlmacenamientoCA = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = cap_almacenamientoCA_dict)
modelo.PDinsatisfecha = pyo.Param(modelo.productos, modelo.periodo_de_tiempo, initialize = productos_dinsatisfecha_dict)

### 6. Variables de decisión 

| Término                 | Descripción                                                                       | Unidades  |
|-------------------------|-----------------------------------------------------------------------------------|-----------|
| Variables de operación  |                                                                               |           |
| $CA_{jt}$               | 1 si se opera un centro de acopio j en el periodo de tiempo t; 0 en caso contrario | [binaria] |
| $AS_{ijt}$              | 1 si se asigna un envío de i a j en el periodo de tiempo t; 0 en caso contrario   | [binaria] |
| Variables de flujo      |                                                                               |           |
| $PA_{pijt}$             | Cantidad del producto p por enviar desde el productor i hasta el centro de acopio j en el periodo de tiempo t | [kg]      |
| $AC_{pjkt}$             | Cantidad del producto p por enviar desde el centro de acopio j al cliente k en el periodo de tiempo t       | [kg]      |
| Variables de inventario |                                                                               |           |
| $InvCA_{pjt}$           | Inventario del producto p en el centro de acopio j en el periodo de tiempo t     | [kg]      |
| Variables demanda insatisfecha |                                                      |           |
| $DInsatisfecha_{pt}$    | Demanda insatisfecha del producto p en el periodo de tiempo t                    | [kg]      |



In [69]:

modelo.CA = pyo.Var(modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.AS = pyo.Var(modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.PA = pyo.Var(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.AC = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.InvCA = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.DInsatisfecha = pyo.Var(modelo.productos, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)


### 7. Restricciones de capacidad y demanda

(2) De Capacidad de productores:
\begin{align*}
& PA_{pijt} = \text{CapProductor}_{pit} \cdot \text{AS}_{ijt} \quad \forall p \in P, \forall i \in I, \forall j \in J, \forall t \in T
\end{align*}

(3) De Flujo:

En los centros de acopio

\begin{align*}
& \sum_{i \in I} PA_{pijt} = \text{InvCA}_{pjt} + \sum_{k \in K} AC_{pjkt} \quad \forall p \in P, \forall j \in J, t=1 \
\\
\\
& \sum_{i \in I} PA_{pijt} + \text{InvCA}_{pjt-1} = \text{InvCA}_{pjt} + \sum_{k \in K} AC_{pjkt} \quad \forall p \in P, \forall j \in J, t \geq 2
\end{align*}

(4) De relación de variables binarias y de Operación:
\begin{align*}
& \sum_{i \in I} PA_{pijt} \leq CA_{jt} \cdot \text{CapAlmacenamientoCA}_{pjt} \quad \forall i \in I, \forall i \in I, \forall p \in P, \forall t \in T
\end{align*}

(5) Asignación:
\begin{align*}
& \sum_{j \in J} AS_{ijt} = 1 \quad \forall j \in J, \forall i \in I, \forall p \in P
\end{align*}

(6) Demanda insatisfecha:
\begin{align*}
& \sum_{j \in J} \sum_{k \in K} AC_{pjkt} + \text{Dinsatisfecha}_{pt} = \sum_{k \in K} \text{DemandaClie}_{pkt} \quad \forall t \in T, \forall p \in P
\end{align*}


In [70]:
def cap_productor_rule(modelo, p, i, j, t):
    return modelo.PA[p, i, j, t] <= modelo.CapProductor[p, i, t] * modelo.AS[i, j, t]


modelo.cap_productor = pyo.Constraint(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=cap_productor_rule)

def flujo_acopio_rule(modelo, p, j, t):
    if t == 1:
        return sum(modelo.PA[p, i, j, t] for i in modelo.productores) == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
    else:
        return (
            sum(modelo.PA[p, i, j, t] for i in modelo.productores)
            + modelo.InvCA[p, j, t - 1]
            == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
        )

modelo.flujo_acopio = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=flujo_acopio_rule)

def binarias_operacion_rule(modelo, p, j, t):
    return sum(modelo.PA[p, i, j, t] for i in modelo.productores) <= modelo.CA[j, t] * modelo.CapAlmacenamientoCA[p, j, t]

modelo.binarias_operacion = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=binarias_operacion_rule)

def binaria_asignacion_rule(modelo, i, t):
    return sum(modelo.AS[i, j, t] for j in modelo.centros_de_acopios) == 1

modelo.binaria_asignacion = pyo.Constraint(modelo.productores, modelo.periodo_de_tiempo, rule=binaria_asignacion_rule)

def demanda_insatisfecha_rule(modelo, p, t):
    return (
        sum(modelo.AC[p, j, k, t] for j in modelo.centros_de_acopios for k in modelo.clientes) + modelo.DInsatisfecha[p, t]
        == sum(modelo.DemandaClie[p, k, t] for k in modelo.clientes)
    )

modelo.demanda_insatisfecha = pyo.Constraint(modelo.productos, modelo.periodo_de_tiempo, rule=demanda_insatisfecha_rule)


### 8. Función objetivo 

FUNCION OBJETIVO
\
\
$Min F$=  $CProducción_{t}$+$COperación_{t}$+$CInventario_{t}$+$CTransporte_{t}$+$DemandaInsatisfecha_{t}$ (1)




\begin{align*}
& CProduccion_t = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} \text{CostoProduccion}_{pit} \cdot \text{PA}_{pijt} \quad \forall t \in T \\
\\
& COperacion = \sum_{j \in J} \text{CostoFijoAcopio}_{jt} \cdot \text{CA}_{jt} + \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} \text{CostoVarAcopio}_{pjt} \cdot \text{PA}_{pijt} \quad \forall t \in T \\
\\
& \text{CInventario}_t = \sum_{j \in J} \sum_{p \in P} \text{CostoInvAcopio}_{pjt} \cdot \text{InvCA}_{pjt} \quad \forall t \in T \\
\\
& \text{CTransporte}_t = \sum_{j \in J} \sum_{i \in I} \sum_{p \in P} \text{CostoTransProdAcopio}_{pijt} \cdot \text{PA}_{pijt} + \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} \text{CostoTransAcopioClie}_{pjkt} \cdot \text{AC}_{pjkt} \quad \forall t \in T \\
\\
& \text{DemandaInsatisfecha}_t = \sum_{p \in P} \text{Dinsatisfecha}_{pt} \cdot \text{PDinsatisfecha}_{pt} \quad \forall t \in T \\
\end{align*}




In [71]:
# Función Objetivo
def objective_rule(modelo):
    return sum(
        modelo.CostoProduccion[p, i, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoFijoAcopio[j, t] * modelo.CA[j, t]
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoVarAcopio[p, j, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoInvAcopio[p, j, t] * modelo.InvCA[p, j, t]
        for p in modelo.productos
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoTransProdAcopio[p, i, j, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoTransAcopioClie[p, j, k, t] * modelo.AC[p, j, k, t]
        for p in modelo.productos
        for j in modelo.centros_de_acopios
        for k in modelo.clientes
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.DInsatisfecha[p, t] * modelo.PDinsatisfecha[p, t]
        for p in modelo.productos
        for t in modelo.periodo_de_tiempo
    )

modelo.objetivo = pyo.Objective(rule=objective_rule, sense=pyo.minimize)


### 9. Resuelve el modelo 

In [72]:
# Resuelve el modelo
# Crear una instancia del modelo
instancia_modelo = modelo.create_instance()

# Resolver el problema
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Después de resolver el modelo
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Imprimir resultados
print("Estado del solver:", solucion.solver.status)
print("Condición de terminación:", solucion.solver.termination_condition)

# Crear un diccionario para almacenar los resultados
resultados = {}


Estado del solver: ok
Condición de terminación: optimal


### 10. Imprime los resultados 

In [73]:
# Imprimir el valor de la función objetivo solo si la solución es óptima
if solucion.solver.status == pyo.SolverStatus.ok and solucion.solver.termination_condition == pyo.TerminationCondition.optimal:
    costo_total = pyo.value(instancia_modelo.objetivo)
    print("Costo total de distribución =", costo_total)
    resultados["CostoTotal"] = costo_total
else:
    print("El modelo no se resolvió de manera óptima. Verifica tu modelo o los datos de entrada.")
    resultados["Estado"] = "No Óptimo"

# Almacenar valores de variables en el diccionario de resultados
for v in instancia_modelo.component_objects(pyo.Var, active=True):
    for index in v:
        resultados[f"Valor_{str(v)}_{index}"] = pyo.value(v[index])

# Guardar resultados en un DataFrame
resultados_df = pd.DataFrame([resultados])

# Guardar resultados en un archivo Excel
resultados_df.to_excel('resultados_modelo_transporte.xlsx', index=False)
print("Resultados guardados en 'resultados_modelo_transporte.xlsx'")

Costo total de distribución = 644177840684.856


Resultados guardados en 'resultados_modelo_transporte.xlsx'


In [74]:
# Después de resolver el modelo
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Imprimir resultados
print("Estado del solver:", solucion.solver.status)
print("Condición de terminación:", solucion.solver.termination_condition)

# Crear un diccionario para almacenar los resultados
resultados = {}

# Imprimir el valor de la función objetivo solo si la solución es óptima
if solucion.solver.status == pyo.SolverStatus.ok and solucion.solver.termination_condition == pyo.TerminationCondition.optimal:
    costo_total = pyo.value(instancia_modelo.objetivo)
    print("Costo total de distribución =", costo_total)
    resultados["CostoTotal"] = costo_total
else:
    print("El modelo no se resolvió de manera óptima. Verifica tu modelo o los datos de entrada.")
    resultados["Estado"] = "No Óptimo"

# Almacenar valores de variables en el diccionario de resultados
for v in instancia_modelo.component_objects(pyo.Var, active=True):
    for index in v:
        resultados[f"Valor_{str(v)}_{index}"] = pyo.value(v[index])

# Guardar resultados en un DataFrame
resultados_df = pd.DataFrame([resultados])

# Guardar resultados en un archivo Excel
resultados_df.to_excel('resultados_modelo_transporte.xlsx', index=False)
print("Resultados guardados en 'resultados_modelo_transporte.xlsx'")



Estado del solver: ok
Condición de terminación: optimal
Costo total de distribución = 644177840684.856
Resultados guardados en 'resultados_modelo_transporte.xlsx'


# SOLUCIÓN DEL MODELO DE TRANSPORTE MEDIANTE ITERACION DEL MODELO

In [75]:

# Instalando biblioteca pyomo y GLPK
# Instalando Pyomo
# pip3 install pyomo
# pip3 install pandas

import time
import pandas as pd
from pyomo.environ import*
import pyomo.environ as pyo

# Crear un modelo abstracto con Pyomo
modelo = pyo.AbstractModel()

# Cargar datos desde archivos CSV
# Asumiendo que tienes archivos CSV llamados 'costo_produccion.csv', 'costo_fijo_acopio.csv', etc.
# Ajusta los nombres de archivos según tus necesidades

productores_df = pd.read_csv('Productores.csv', index_col='Productores')
centros_de_acopios_df = pd.read_csv('CAcopios.csv', index_col='CAcopios')
clientes_df = pd.read_csv('Clientes.csv', index_col='Clientes')
productos_df = pd.read_csv('Productos.csv', index_col='Productos')
periodo_de_tiempo_df = pd.read_csv('Tiempo.csv', index_col='Tiempo')

costo_produccion_df = pd.read_csv('CostoProduccion.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_fijo_acopio_df = pd.read_csv('CostoFijoAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_var_acopio_df = pd.read_csv('CostoVarAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_inv_acopio_df = pd.read_csv('CostoInvAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_prod_acopio_df = pd.read_csv('CostoTransProdAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_acopio_clie_df = pd.read_csv('CostoTransAcopioClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
demanda_clie_df = pd.read_csv('DemandaClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig') 
cap_productor_df = pd.read_csv('CapProductor.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
cap_almacenamientoCA_df = pd.read_csv('CapAlmacenamientoCA.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
productos_dinsatisfecha_df = pd.read_csv('PDinsatisfecha.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  

# Convertir los DataFrames a diccionarios
costo_produccion_dict = {(row['Productos'], row['Productores'], row['Tiempo']): row['CostoProduccion'] for _, row in costo_produccion_df.iterrows()}
costo_fijo_acopio_dict = {(row['CAcopios'], row['Tiempo']): row['CostoFijoAcopio'] for _, row in costo_fijo_acopio_df.iterrows()}
costo_var_acopio_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CostoVarAcopio'] for _, row in costo_var_acopio_df.iterrows()}
costo_inv_acopio_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CostoInvAcopio'] for _, row in costo_inv_acopio_df.iterrows()}
costo_trans_prod_acopio_dict = {(row['Productos'], row['Productores'], row['CAcopios'], row['Tiempo']): row['CostoTransProdAcopio'] for _, row in costo_trans_prod_acopio_df.iterrows()}
costo_trans_acopio_clie_dict = {(row['Productos'], row['Productores'], row['Clientes'], row['Tiempo']): row['CostoTransAcopioClie'] for _, row in costo_trans_acopio_clie_df.iterrows()}
demanda_clie_dict = {(row['Productos'], row['Clientes'], row['Tiempo']): row['DemandaClie']  for _, row in demanda_clie_df.iterrows()}
cap_productor_dict = {(row['Productos'], row['Productores'], row['Tiempo']): row['CapProductor']  for _, row in cap_productor_df.iterrows()}
cap_almacenamientoCA_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CapAlmacenamientoCA']  for _, row in cap_almacenamientoCA_df.iterrows()}
productos_dinsatisfecha_dict = {(row['Productos'], row['Tiempo']): row['PDinsatisfecha']  for _, row in productos_dinsatisfecha_df.iterrows()}

modelo.productores = pyo.Set(initialize=productores_df.index)  # Conjunto I de productores
modelo.centros_de_acopios = pyo.Set(initialize=centros_de_acopios_df.index)  # Conjunto J de centros de acopio
modelo.clientes = pyo.Set(initialize=clientes_df.index)  # Conjunto K de clientes
modelo.productos = pyo.Set(initialize=productos_df.index)  # Conjunto P de productos
modelo.periodo_de_tiempo = pyo.Set(initialize=periodo_de_tiempo_df.index)  # Conjunto T de periodos de tiempo


modelo.CostoProduccion = pyo.Param(modelo.productos, modelo.productores, modelo.periodo_de_tiempo, initialize = costo_produccion_dict)
modelo.CostoFijoAcopio = pyo.Param(modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_fijo_acopio_dict)
modelo.CostoVarAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_var_acopio_dict)
modelo.CostoInvAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_inv_acopio_dict)
modelo.CostoTransProdAcopio = pyo.Param(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_trans_prod_acopio_dict)
modelo.CostoTransAcopioClie = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, initialize = costo_trans_acopio_clie_dict)
modelo.DemandaClie = pyo.Param(modelo.productos, modelo.clientes, modelo.periodo_de_tiempo, initialize = demanda_clie_dict)
modelo.CapProductor = pyo.Param(modelo.productos, modelo.productores, modelo.periodo_de_tiempo, initialize = cap_productor_dict)
modelo.CapAlmacenamientoCA = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = cap_almacenamientoCA_dict)
modelo.PDinsatisfecha = pyo.Param(modelo.productos, modelo.periodo_de_tiempo, initialize = productos_dinsatisfecha_dict)


modelo.CA = pyo.Var(modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.AS = pyo.Var(modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.PA = pyo.Var(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.AC = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.InvCA = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.DInsatisfecha = pyo.Var(modelo.productos, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)


def cap_productor_rule(modelo, p, i, j, t):
    return modelo.PA[p, i, j, t] <= modelo.CapProductor[p, i, t] * modelo.AS[i, j, t]


modelo.cap_productor = pyo.Constraint(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=cap_productor_rule)

def flujo_acopio_rule(modelo, p, j, t):
    if t == 1:
        return sum(modelo.PA[p, i, j, t] for i in modelo.productores) == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
    else:
        return (
            sum(modelo.PA[p, i, j, t] for i in modelo.productores)
            + modelo.InvCA[p, j, t - 1]
            == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
        )

modelo.flujo_acopio = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=flujo_acopio_rule)

def binarias_operacion_rule(modelo, p, j, t):
    return sum(modelo.PA[p, i, j, t] for i in modelo.productores) <= modelo.CA[j, t] * modelo.CapAlmacenamientoCA[p, j, t]

modelo.binarias_operacion = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=binarias_operacion_rule)

def asignacion_rule(modelo, i, p):
    return sum(modelo.AS[i, j, t] for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) == 1

modelo.asignacion = pyo.Constraint(modelo.productores, modelo.productos, rule=asignacion_rule)

def demanda_insatisfecha_rule(modelo, p, t):
    return (
        sum(modelo.AC[p, j, k, t] for j in modelo.centros_de_acopios for k in modelo.clientes) + modelo.DInsatisfecha[p, t]
        == sum(modelo.DemandaClie[p, k, t] for k in modelo.clientes)
    )

modelo.demanda_insatisfecha = pyo.Constraint(modelo.productos, modelo.periodo_de_tiempo, rule=demanda_insatisfecha_rule)

def objective_rule(modelo):
    return sum(
        modelo.CostoProduccion[p, i, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoFijoAcopio[j, t] * modelo.CA[j, t]
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoVarAcopio[p, j, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoInvAcopio[p, j, t] * modelo.InvCA[p, j, t]
        for p in modelo.productos
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoTransProdAcopio[p, i, j, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoTransAcopioClie[p, j, k, t] * modelo.AC[p, j, k, t]
        for p in modelo.productos
        for j in modelo.centros_de_acopios
        for k in modelo.clientes
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.DInsatisfecha[p, t] * modelo.PDinsatisfecha[p, t]
        for p in modelo.productos
        for t in modelo.periodo_de_tiempo
    )

modelo.objetivo = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Resuelve el modelo
# Crear una instancia del modelo
instancia_modelo = modelo.create_instance()



# Crear una instancia del modelo
instancia_modelo = modelo.create_instance()

# Iniciar el temporizador
inicio_tiempo = time.time()


# Resolver el problema
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Después de resolver el modelo
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Detener el temporizador
fin_tiempo = time.time()

# Imprimir el tiempo transcurrido
tiempo_transcurrido = fin_tiempo - inicio_tiempo
print(f"Tiempo transcurrido: {tiempo_transcurrido} segundos")



# Imprimir resultados
print("Estado del solver:", solucion.solver.status)
print("Condición de terminación:", solucion.solver.termination_condition)

# Crear un diccionario para almacenar los resultados
resultados = {}

# Imprimir el valor de la función objetivo solo si la solución es óptima
if solucion.solver.status == pyo.SolverStatus.ok and solucion.solver.termination_condition == pyo.TerminationCondition.optimal:
    costo_total = pyo.value(instancia_modelo.objetivo)
    print("Costo total de distribución =", costo_total)
    resultados["CostoTotal"] = costo_total
else:
    print("El modelo no se resolvió de manera óptima. Verifica tu modelo o los datos de entrada.")
    resultados["Estado"] = "No Óptimo"

# Almacenar valores de variables en el diccionario de resultados
for v in instancia_modelo.component_objects(pyo.Var, active=True):
    for index in v:
        resultados[f"Valor_{str(v)}_{index}"] = pyo.value(v[index])

# Guardar resultados en un DataFrame
resultados_df = pd.DataFrame([resultados])

# Guardar resultados en un archivo Excel
resultados_df.to_excel('resultados_modelo_transporte.xlsx', index=False)
print("Resultados guardados en 'resultados_modelo_transporte.xlsx'")






Tiempo transcurrido: 2.5445516109466553 segundos
Estado del solver: ok
Condición de terminación: optimal
Costo total de distribución = 1218978808150.0
Resultados guardados en 'resultados_modelo_transporte.xlsx'


In [76]:

# Instalando biblioteca pyomo y GLPK
# Instalando Pyomo
# pip3 install pyomo
# pip3 install pandas

import time
import pandas as pd
from pyomo.environ import*
import pyomo.environ as pyo

# Crear un modelo abstracto con Pyomo
modelo = pyo.AbstractModel()

# Cargar datos desde archivos CSV
# Asumiendo que tienes archivos CSV llamados 'costo_produccion.csv', 'costo_fijo_acopio.csv', etc.
# Ajusta los nombres de archivos según tus necesidades

productores_df = pd.read_csv('Productores.csv', index_col='Productores')
centros_de_acopios_df = pd.read_csv('CAcopios.csv', index_col='CAcopios')
clientes_df = pd.read_csv('Clientes.csv', index_col='Clientes')
productos_df = pd.read_csv('Productos.csv', index_col='Productos')
periodo_de_tiempo_df = pd.read_csv('Tiempo.csv', index_col='Tiempo')

costo_produccion_df = pd.read_csv('CostoProduccion.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_fijo_acopio_df = pd.read_csv('CostoFijoAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_var_acopio_df = pd.read_csv('CostoVarAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_inv_acopio_df = pd.read_csv('CostoInvAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_prod_acopio_df = pd.read_csv('CostoTransProdAcopio.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
costo_trans_acopio_clie_df = pd.read_csv('CostoTransAcopioClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
demanda_clie_df = pd.read_csv('DemandaClie.csv', delimiter=';', decimal=',', encoding='utf-8-sig') 
cap_productor_df = pd.read_csv('CapProductor.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
cap_almacenamientoCA_df = pd.read_csv('CapAlmacenamientoCA.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  
productos_dinsatisfecha_df = pd.read_csv('PDinsatisfecha.csv', delimiter=';', decimal=',', encoding='utf-8-sig')  

# Convertir los DataFrames a diccionarios
costo_produccion_dict = {(row['Productos'], row['Productores'], row['Tiempo']): row['CostoProduccion'] for _, row in costo_produccion_df.iterrows()}
costo_fijo_acopio_dict = {(row['CAcopios'], row['Tiempo']): row['CostoFijoAcopio'] for _, row in costo_fijo_acopio_df.iterrows()}
costo_var_acopio_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CostoVarAcopio'] for _, row in costo_var_acopio_df.iterrows()}
costo_inv_acopio_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CostoInvAcopio'] for _, row in costo_inv_acopio_df.iterrows()}
costo_trans_prod_acopio_dict = {(row['Productos'], row['Productores'], row['CAcopios'], row['Tiempo']): row['CostoTransProdAcopio'] for _, row in costo_trans_prod_acopio_df.iterrows()}
costo_trans_acopio_clie_dict = {(row['Productos'], row['Productores'], row['Clientes'], row['Tiempo']): row['CostoTransAcopioClie'] for _, row in costo_trans_acopio_clie_df.iterrows()}
demanda_clie_dict = {(row['Productos'], row['Clientes'], row['Tiempo']): row['DemandaClie']  for _, row in demanda_clie_df.iterrows()}
cap_productor_dict = {(row['Productos'], row['Productores'], row['Tiempo']): row['CapProductor']  for _, row in cap_productor_df.iterrows()}
cap_almacenamientoCA_dict = {(row['Productos'], row['CAcopios'], row['Tiempo']): row['CapAlmacenamientoCA']  for _, row in cap_almacenamientoCA_df.iterrows()}
productos_dinsatisfecha_dict = {(row['Productos'], row['Tiempo']): row['PDinsatisfecha']  for _, row in productos_dinsatisfecha_df.iterrows()}

modelo.productores = pyo.Set(initialize=productores_df.index)  # Conjunto I de productores
modelo.centros_de_acopios = pyo.Set(initialize=centros_de_acopios_df.index)  # Conjunto J de centros de acopio
modelo.clientes = pyo.Set(initialize=clientes_df.index)  # Conjunto K de clientes
modelo.productos = pyo.Set(initialize=productos_df.index)  # Conjunto P de productos
modelo.periodo_de_tiempo = pyo.Set(initialize=periodo_de_tiempo_df.index)  # Conjunto T de periodos de tiempo


modelo.CostoProduccion = pyo.Param(modelo.productos, modelo.productores, modelo.periodo_de_tiempo, initialize = costo_produccion_dict)
modelo.CostoFijoAcopio = pyo.Param(modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_fijo_acopio_dict)
modelo.CostoVarAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_var_acopio_dict)
modelo.CostoInvAcopio = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_inv_acopio_dict)
modelo.CostoTransProdAcopio = pyo.Param(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = costo_trans_prod_acopio_dict)
modelo.CostoTransAcopioClie = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, initialize = costo_trans_acopio_clie_dict)
modelo.DemandaClie = pyo.Param(modelo.productos, modelo.clientes, modelo.periodo_de_tiempo, initialize = demanda_clie_dict)
modelo.CapProductor = pyo.Param(modelo.productos, modelo.productores, modelo.periodo_de_tiempo, initialize = cap_productor_dict)
modelo.CapAlmacenamientoCA = pyo.Param(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, initialize = cap_almacenamientoCA_dict)
modelo.PDinsatisfecha = pyo.Param(modelo.productos, modelo.periodo_de_tiempo, initialize = productos_dinsatisfecha_dict)


modelo.CA = pyo.Var(modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.AS = pyo.Var(modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.Binary)
modelo.PA = pyo.Var(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.AC = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.clientes, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.InvCA = pyo.Var(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)
modelo.DInsatisfecha = pyo.Var(modelo.productos, modelo.periodo_de_tiempo, within=pyo.NonNegativeReals)


def cap_productor_rule(modelo, p, i, j, t):
    return modelo.PA[p, i, j, t] <= modelo.CapProductor[p, i, t] * modelo.AS[i, j, t]


modelo.cap_productor = pyo.Constraint(modelo.productos, modelo.productores, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=cap_productor_rule)

def flujo_acopio_rule(modelo, p, j, t):
    if t == 1:
        return sum(modelo.PA[p, i, j, t] for i in modelo.productores) == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
    else:
        return (
            sum(modelo.PA[p, i, j, t] for i in modelo.productores)
            + modelo.InvCA[p, j, t - 1]
            == modelo.InvCA[p, j, t] + sum(modelo.AC[p, j, k, t] for k in modelo.clientes)
        )

modelo.flujo_acopio = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=flujo_acopio_rule)

def binarias_operacion_rule(modelo, p, j, t):
    return sum(modelo.PA[p, i, j, t] for i in modelo.productores) <= modelo.CA[j, t] * modelo.CapAlmacenamientoCA[p, j, t]

modelo.binarias_operacion = pyo.Constraint(modelo.productos, modelo.centros_de_acopios, modelo.periodo_de_tiempo, rule=binarias_operacion_rule)

def asignacion_rule(modelo, i, p):
    return sum(modelo.AS[i, j, t] for j in modelo.centros_de_acopios for t in modelo.periodo_de_tiempo) == 1

modelo.asignacion = pyo.Constraint(modelo.productores, modelo.productos, rule=asignacion_rule)

def demanda_insatisfecha_rule(modelo, p, t):
    return (
        sum(modelo.AC[p, j, k, t] for j in modelo.centros_de_acopios for k in modelo.clientes) + modelo.DInsatisfecha[p, t]
        == sum(modelo.DemandaClie[p, k, t] for k in modelo.clientes)
    )

modelo.demanda_insatisfecha = pyo.Constraint(modelo.productos, modelo.periodo_de_tiempo, rule=demanda_insatisfecha_rule)

def objective_rule(modelo):
    return sum(
        modelo.CostoProduccion[p, i, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoFijoAcopio[j, t] * modelo.CA[j, t]
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoVarAcopio[p, j, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoInvAcopio[p, j, t] * modelo.InvCA[p, j, t]
        for p in modelo.productos
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoTransProdAcopio[p, i, j, t] * modelo.PA[p, i, j, t]
        for p in modelo.productos
        for i in modelo.productores
        for j in modelo.centros_de_acopios
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.CostoTransAcopioClie[p, j, k, t] * modelo.AC[p, j, k, t]
        for p in modelo.productos
        for j in modelo.centros_de_acopios
        for k in modelo.clientes
        for t in modelo.periodo_de_tiempo
    ) + sum(
        modelo.DInsatisfecha[p, t] * modelo.PDinsatisfecha[p, t]
        for p in modelo.productos
        for t in modelo.periodo_de_tiempo
    )

modelo.objetivo = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Resolver el problema
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Después de resolver el modelo
solucion = pyo.SolverFactory('glpk').solve(instancia_modelo)

# Imprimir el tiempo transcurrido
tiempo_transcurrido = fin_tiempo - inicio_tiempo
print(f"Tiempo transcurrido: {tiempo_transcurrido} segundos")

# Imprimir resultados
print("Estado del solver:", solucion.solver.status)
print("Condición de terminación:", solucion.solver.termination_condition)

# Crear un diccionario para almacenar los resultados de variables en DataFrames
resultados_dataframes = {}

# Imprimir valores de las variables de decisión y almacenarlos en DataFrames
for v in instancia_modelo.component_objects(pyo.Var, active=True):
    indices = []
    valores = []
    for index in v:
        indices.append(index)
        valores.append(pyo.value(v[index]))

    resultados_df = pd.DataFrame({f"{str(v)}_{index}": valor for index, valor in zip(indices, valores)}, index=[0])
    resultados_dataframes[str(v)] = resultados_df

    # Imprimir el DataFrame con los valores de la variable
    print(f"\nValores de la variable {str(v)}:")
    print(resultados_df)

# Guardar DataFrames de resultados en archivos Excel
for nombre_variable, df_resultado in resultados_dataframes.items():
    df_resultado.to_excel(f'resultados_{nombre_variable}.xlsx', index=False)
    print(f"Resultados de {nombre_variable} guardados en 'resultados_{nombre_variable}.xlsx'")

Tiempo transcurrido: 2.5445516109466553 segundos
Estado del solver: ok
Condición de terminación: optimal

Valores de la variable CA:
   CA_('SAN JUAN DEL CESAR', 1)  CA_('SAN JUAN DEL CESAR', 2)  \
0                           0.0                           1.0   

   CA_('SAN JUAN DEL CESAR', 3)  CA_('SAN JUAN DEL CESAR', 4)  \
0                           0.0                           0.0   

   CA_('SAN JUAN DEL CESAR', 5)  CA_('SAN JUAN DEL CESAR', 6)  \
0                           0.0                           0.0   

   CA_('SAN JUAN DEL CESAR', 7)  CA_('SAN JUAN DEL CESAR', 8)  \
0                           0.0                           0.0   

   CA_('SAN JUAN DEL CESAR', 9)  CA_('SAN JUAN DEL CESAR', 10)  ...  \
0                           0.0                            0.0  ...   

   CA_('RIOHACHA', 3)  CA_('RIOHACHA', 4)  CA_('RIOHACHA', 5)  \
0                 0.0                 0.0                 0.0   

   CA_('RIOHACHA', 6)  CA_('RIOHACHA', 7)  CA_('RIOHACHA', 8)  \
0   